### puzzle问题

可以用图表示状态转换。考虑一个名为`puzzle`的游戏，游戏给出一个三乘三棋盘，棋盘上摆放了8个数字正方形，棋盘上有一个空位上没有摆放数字正方形，希望从某一状态转换到另一状态。可以将棋盘的一种状态视作一个节点，而边则表示不同节点之间的合法转换，因此可以用搜索的办法解决这个问题。

首先看棋盘的表示。

In [1]:
class puzzle(object):
    #order: 初始状态
    #spot: 棋盘空位
    def __init__(self, order):
        self.label = order
        for index in range(9):
            if order[index] == '0':
                self.spot = index
                return None
    #让self.label[to]变为空位
    def transition(self, to):
        label = self.label
        blankLocation = self.spot
        newBlankLabel = str(label[to])
        newLabel = ''#用于存储新状态
        for i in range(9):
            if i == to:
                newLabel += '0'
            elif i == blankLocation:
                newLabel += newBlankLabel
            else:
                newLabel += str(label[i])
        return puzzle(newLabel)#返回新状态（棋盘布局）
    def __str__(self):
        return self.label

结合下图，`shiftDict`应该如此：

| 0 | 1 | 2 |
|---|---|---|
| 3 | 4 | 5 |
| 6 | 7 | 8 |

In [2]:
shiftDict = {}
shiftDict[0] = [1, 3]
shiftDict[1] = [0, 2, 4]
shiftDict[2] = [1, 5]
shiftDict[3] = [0, 4, 6]
shiftDict[4] = [1, 3, 5, 7]
shiftDict[5] = [2, 4, 8]
shiftDict[6] = [3, 7]
shiftDict[7] = [4, 6, 8]
shiftDict[8] = [5, 7]

实现一个简单版本的先宽搜索，用于求解`puzzle`问题。

In [3]:
def notInPath(node, path):
    for elt in path:
        if node.label == elt.label:
            return False
    return True

In [4]:
def BFSWithGenerator(start, end, q = []):
    
    initPath = [start]
    q.append(initPath)
    while len(q) != 0:
        tmpPath = q.pop(0)
        lastNode = tmpPath[len(tmpPath) - 1]
        if lastNode.label == end.label:
            return tmpPath
        for shift in shiftDict[lastNode.spot]:
            new = lastNode.transition(shift)
            if notInPath(new, tmpPath):
                newPath = tmpPath + [new]
                q.append(newPath)
    return None

显然也可用先深搜索处理这个问题。这里的DFS与前面的BFS只有一个不同：

- BFS中，插入`queue`的尾部，而在DFS中，插入`stack`的头部，相当于“`queue`”的头部。对`stack`进行`pop`操作实际上是取出其最顶的元素。

In [6]:
def DFSWithGenerator(start, end, stack = []):
    #assumes graph is a Digraph
    #assumes start and end are nodes in graph
    initPath = [start]
    stack.insert(0, initPath)
    while len(stack)!= 0:
        tmpPath = stack.pop(0)
        lastNode = tmpPath[len(tmpPath) - 1]
        if lastNode.label == end.label:
            return tmpPath
        for shift in shiftDict[lastNode.spot]:
            new = lastNode.transition(shift)
            if notInPath(new, tmpPath): #avoid cycles
                newPath = tmpPath + [new]
                stack.insert(0, newPath)
    return None

配上数据开始跑。

In [7]:
goal = puzzle('012345678')
test1 = puzzle('125638047')

In [8]:
def printGrid(pzl):
    data = pzl.label
    print data[0], data[1], data[2]
    print data[3], data[4], data[5]
    print data[6], data[7], data[8]
    print ''

def printSolution(path):
    for elt in path:
        printGrid(elt)

In [11]:
path = BFSWithGenerator(test1, goal)
printSolution(path)

1 2 5
6 3 8
0 4 7

1 2 5
6 3 8
4 0 7

1 2 5
6 0 8
4 3 7

1 2 5
0 6 8
4 3 7

1 2 5
4 6 8
0 3 7

1 2 5
4 6 8
3 0 7

1 2 5
4 0 8
3 6 7

1 2 5
0 4 8
3 6 7

1 2 5
3 4 8
0 6 7

1 2 5
3 4 8
6 0 7

1 2 5
3 4 8
6 7 0

1 2 5
3 4 0
6 7 8

1 2 0
3 4 5
6 7 8

1 0 2
3 4 5
6 7 8

0 1 2
3 4 5
6 7 8



而用先深搜索则根本跑不出结果。

### Maximum Cliques

最大子团问题。子团，即一个完全连通分量，指一个子图，图中任意两个节点之间都存在连接。子团具有许多现实对应物，比如疾病感染人群，或朋友圈。

这里只讨论用暴力方法求解最大子团问题。先考虑用递归的方法求解幂集。随后求解图的幂集。然后遍历幂集中的元素，考察它们之间的连线在我们要研究的图中是否存在。

幂集。

In [10]:
def powerSet(elts):
    if len(elts) == 0:
        return [[]]
    else:
        smaller = powerSet(elts[1:])
        elt = [elts[0]]
        withElt = []
        for s in smaller:
            withElt.append(s + elt)
        allofthem = smaller + withElt
        return allofthem

In [2]:
powerSet([1,2,3])

[[], [3], [2], [3, 2], [1], [3, 1], [2, 1], [3, 2, 1]]

图的幂集。

In [8]:
def powerGraph(gr):
    nodes = gr.nodes
    nodesList = []
    for elt in nodes:
        nodesList.append(elt)
    pSet = powerSet(nodesList)
    return pSet

节点之间的连线全部存在吗？`candidate`是图的幂集中的一个元素，是一堆点。

In [4]:
def allConnected(gr, candidate):
    for n in candidate:
        for m in candidate:
            if not n == m:
                if n not in gr.childrenOf(m):
                    return False
    return True

最大子团。

In [12]:
def maxClique(gr):
    candidates = powerGraph(gr)
    keepEm = []
    for candidate in candidates:
        if allConnected(gr, candidate):
            keepEm.append(candidate)
    bestLength = 0
    bestSoln = None
    for test in keepEm:
        if len(test) > bestLength:
            bestLength = len(test)
            bestSoln = test
    return bestSoln

测试。

In [6]:
class Node(object):
    def __init__(self, name):
        self.name = str(name)
    def getName(self):
        return self.name
    def __str__(self):
        return self.name

class Edge(object):
    def __init__(self, src, dest):
        self.src = src
        self.dest = dest
    def getSource(self):
        return self.src
    def getDestination(self):
        return self.dest
    def __str__(self):
        return str(self.src) + '->' + str(self.dest)

class WeightedEdge(Edge):
    def __init__(self, src, dest, weight = 1.0):
        self.src = src
        self.dest = dest
        self.weight = weight
    def getWeight(self):
        return self.weight
    def __str__(self):
        return str(self.src) + '->(' + str(self.weight) + ')'\
            + str(self.dest)

class Digraph(object):
    def __init__(self):
        self.nodes = set([])
        self.edges = {}
    def addNode(self, node):
        if node in self.nodes:
            raise ValueError('Duplicate node')
        else:
            self.nodes.add(node)
            self.edges[node] = []
    def addEdge(self, edge):
        src = edge.getSource()
        dest = edge.getDestination()
        if not(src in self.nodes and dest in self.nodes):
            raise ValueError('Node not in graph')
        self.edges[src].append(dest)
    def childrenOf(self, node):
        return self.edges[node]
    def hasNode(self, node):
        return node in self.nodes
    def __str__(self):
        res = ''
        for k in self.edges:
            for d in self.edges[k]:
                res = res + str(k) + '->' + str(d) + '\n'
        return res[:-1]

class Graph(Digraph):
    def addEdge(self, edge):
        Digraph.addEdge(self, edge)
        rev = Edge(edge.getDestination(), edge.getSource())
        Digraph.addEdge(self, rev)

In [13]:
def testGraph():
    nodes = []
    for name in range(5):
        nodes.append(Node(str(name)))
    g = Graph()
    for n in nodes:
        g.addNode(n)
    g.addEdge(Edge(nodes[0],nodes[1]))
    g.addEdge(Edge(nodes[1],nodes[2]))
    g.addEdge(Edge(nodes[2],nodes[0]))
    g.addEdge(Edge(nodes[2],nodes[4]))
    g.addEdge(Edge(nodes[4],nodes[3]))
    return g


trialGraph = testGraph()
myClique = maxClique(trialGraph)

最大子团包含三个节点。

In [14]:
myClique